## 103590450 四資四 馬茂源

In [32]:
from pyspark import SparkConf, SparkContext, SQLContext
#from pyspark.sql.functions import mean, max as max_, min as min_, stddev, count
import os, math

In [33]:
conf = (SparkConf()
        #.setMaster('spark://10.100.5.182:7077')
        .setMaster("local")
        .setAppName("hw1"))

In [34]:
sc = SparkContext(conf=conf)
sql_sc = SQLContext(sc)

In [35]:
# rdd = (sc.textFile('./household_power_consumption.csv')
#        .map(lambda line: line.split(';')))
# header = rdd.first()
# data = sql_sc.createDataFrame(rdd.filter(lambda line: line != header), header)
# data.show()

In [36]:
# data = pd.read_csv('./household_power_consumption.csv', 
#                    delimiter=';', 
#                    dtype={'Date':str,
#                           'Time':str,
#                           'Global_active_power':str,
#                           'Global_reactive_power':str,
#                           'Voltage':str,
#                           'Global_intensity':str,
#                           'Sub_metering_1':str,
#                           'Sub_metering_2':str,
#                           'sub_metering_3':str,})
# data = sql_sc.createDataFrame(data)

In [37]:
data = sql_sc.read.csv('./household_power_consumption.csv', sep=';', header=True)

In [38]:
data = data.drop('Date', 'Time', 'Sub_metering_1', 
        'Sub_metering_2', 
        'Sub_metering_3')

In [39]:
data = (data.withColumn('Global_active_power', data.Global_active_power.cast('float'))
        .withColumn('Global_reactive_power', data.Global_reactive_power.cast('float'))
        .withColumn('Voltage', data.Voltage.cast('float'))
        .withColumn('Global_intensity', data.Global_intensity.cast('float')))

In [40]:
data = data.na.drop()

1. Output the **minimum**, **maximum**, and **count** of the columns:`Global_active_power`, `Global_reactive_power`, `Voltage`, and `Global_intensity`
2. Output the **mean** and **standard deviation** of these columns


In [41]:
features = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']

In [42]:
# def get_count(df, feature):
#     return df.select(feature).count()
# def get_minmax(df, feature):
#     return (data.select(feature)
#             .rdd
#             .sortBy(lambda x: x, ascending=False))

In [43]:
count = data.count()

In [44]:
summary = {k:{} for k in ['min', 'max', 'mean', 'stddev', 'count']}
summary['count'] = {f:count for f in features}

In [57]:
summary['max'] = data.rdd.max().asDict()
summary['min'] = data.rdd.min().asDict()

In [61]:
data.select(rdd.max()

Row(Global_active_power=11.121999740600586, Global_reactive_power=0.17399999499320984, Voltage=229.77999877929688, Global_intensity=48.400001525878906)

In [15]:
sum_ = (data.rdd.map(lambda x: (x.Global_active_power, 
                        x.Global_reactive_power,
                        x.Voltage,
                        x.Global_intensity))
        .reduce(lambda x,y: tuple(x_n+y_n for x_n, y_n in zip(x, y))))

In [16]:
for i, f in enumerate(features):
    summary['mean'][f] = sum_[i]/summary['count'][f]

In [17]:
temp = (data.rdd.map(lambda x: ((x.Global_active_power-summary['mean']['Global_active_power'])**2, 
                        (x.Global_reactive_power-summary['mean']['Global_reactive_power'])**2,
                        (x.Voltage-summary['mean']['Voltage'])**2,
                        (x.Global_intensity-summary['mean']['Global_intensity'])**2))
        .reduce(lambda x, y: tuple(x_n+y_n for x_n, y_n in zip(x, y))))

for i, f in enumerate(features):
    summary['stddev'][f] = math.sqrt(temp[i] / (summary['count'][f]-1)) 

{'count': {'Global_active_power': 2049280,
  'Global_intensity': 2049280,
  'Global_reactive_power': 2049280,
  'Voltage': 2049280},
 'max': {'Global_active_power': 11.121999740600586,
  'Global_intensity': 48.400001525878906,
  'Global_reactive_power': 0.17399999499320984,
  'Voltage': 229.77999877929688},
 'mean': {'Global_active_power': 1.0916150366540094,
  'Global_intensity': 4.627759313004169,
  'Global_reactive_power': 0.1237144765251571,
  'Voltage': 240.83985796672414},
 'min': {'Global_active_power': 0.07599999755620956,
  'Global_intensity': 0.20000000298023224,
  'Global_reactive_power': 0.0,
  'Voltage': 233.9199981689453},
 'stddev': {'Global_active_power': 1.0572941611196849,
  'Global_intensity': 4.444396258970011,
  'Global_reactive_power': 0.11272197958677589,
  'Voltage': 3.239986661205925}}

In [58]:
summary

{'count': {'Global_active_power': 2049280,
  'Global_intensity': 2049280,
  'Global_reactive_power': 2049280,
  'Voltage': 2049280},
 'max': {'Global_active_power': 11.121999740600586,
  'Global_intensity': 48.400001525878906,
  'Global_reactive_power': 0.17399999499320984,
  'Voltage': 229.77999877929688},
 'mean': {},
 'min': {'Global_active_power': 0.07599999755620956,
  'Global_intensity': 0.20000000298023224,
  'Global_reactive_power': 0.0,
  'Voltage': 233.9199981689453},
 'stddev': {}}

In [18]:
# summary = data.select(features).describe()
# summary.show()
# +-------+-------------------+---------------------+------------------+-----------------+
# |summary|Global_active_power|Global_reactive_power|           Voltage| Global_intensity|
# +-------+-------------------+---------------------+------------------+-----------------+
# |  count|            2049280|              2049280|           2049280|          2049280|
# |   mean| 1.0916150366540094|   0.1237144765251571|240.83985796672414|4.627759313004169|
# | stddev| 1.0572941611180013|  0.11272197958641254|  3.23998666120589|4.444396258981289|
# |    min|              0.076|                  0.0|             223.2|              0.2|
# |    max|             11.122|                 1.39|            254.15|             48.4|
# +-------+-------------------+---------------------+------------------+-----------------+

3 . Perform **min-max normalization** on the columns to generate normalized output

In [28]:
for f in features:
    min_ = summary['min'][f]
    max_ = summary['max'][f] 
    data = (data.withColumn('norm_' + f, 
                     (data[f] - min_) / (max_ - min_)))

In [29]:
data.select(['norm_'+f for f in features]).show()

+------------------------+--------------------------+--------------------+---------------------+
|norm_Global_active_power|norm_Global_reactive_power|        norm_Voltage|norm_Global_intensity|
+------------------------+--------------------------+--------------------+---------------------+
|      0.3747963225473887|         2.402298992321791| -0.2222218127001795|  0.37759334108949466|
|     0.47836323183743595|         2.505747138249038| 0.07004669779853236|  0.47302903063577767|
|     0.47963065346431577|         2.862069024578524| 0.15217511490164715|  0.47302903063577767|
|     0.48089807509119564|        2.8850574062969567|0.043476498144250864|  0.47302903063577767|
|     0.32500452429198895|         3.034482829494229| -0.4251195087701193|   0.3236514459515824|
|       0.311787077990743|          3.00000017127772| -0.2657019965428149|  0.30705393214538884|
|      0.3282636238926484|        2.9885057235019237| -0.2826082950327843|   0.3236514459515824|
|      0.3280825759940403|    

In [30]:
sc.stop()